In [0]:
# Célula 1: Carregar o Motor de Decisão (Modelo Gold)
from pyspark.sql.functions import col, when, lit

# Lendo a tabela onde salvamos os scores de risco
df_scores = spark.read.table("portfolio_credit.german_credit_gold_predictions")

print("Scores carregados. Preparando motor de regras...")
# Vamos ver alguns exemplos de scores altos e baixos
display(df_scores.select("score_risco", "prediction", "credit_amount", "duration").limit(5))

In [0]:
# Célula 2: Definindo a Estratégia (Decision Engine)

# Criamos colunas de Ação (NBA) e Oferta (NBO) baseadas no Score do modelo
# Note como usamos a variável 'credit_amount' também, já que ela foi importante no seu modelo!

df_strategy = df_scores.withColumn(
    "segmento_risco",
    when(col("score_risco") < 0.3, "1. Baixo Risco (VIP)")
    .when(col("score_risco") < 0.6, "2. Risco Médio")
    .otherwise("3. Alto Risco")
).withColumn(
    "decisao_final",
    when(col("score_risco") > 0.6, "NEGAR") # Se risco alto, nega
    .otherwise("APROVAR")
).withColumn(
    "nbo_oferta_recomendada", # Next Best Offer
    when(col("score_risco") < 0.3, "Cartão Platinum + Taxa Reduzida")
    .when((col("score_risco") >= 0.3) & (col("credit_amount") > 5000), "Seguro Prestamista") # Cross-sell
    .otherwise("Empréstimo Padrão")
).withColumn(
    "nba_acao_mitigacao", # Next Best Action
    when(col("score_risco") > 0.6, "Solicitar Avalista ou Penhor")
    .when(col("score_risco") > 0.4, "Ligar para confirmar dados")
    .otherwise("Aprovação Automática")
)

# Selecionar colunas para visualização gerencial
cols_show = ["score_risco", "segmento_risco", "decisao_final", "nbo_oferta_recomendada", "nba_acao_mitigacao", "credit_amount"]

print("Estratégia aplicada com sucesso.")
display(df_strategy.select(cols_show).limit(10))

In [0]:
# Célula 3: Business Value (Quanto economizamos?)

# Cenário 1: Sem modelo (Aprovamos tudo)
# Vamos supor que cada 'label=1' (Mau Pagador) nos dá um prejuízo médio de 30% do valor do empréstimo
total_prejuizo_sem_modelo = df_scores.filter(col("label") == 1).selectExpr("sum(credit_amount * 0.3)").collect()[0][0]

# Cenário 2: Com SEU modelo
# Nós negamos crédito para quem tem score > 0.6. 
# Quanto deixamos de perder? (Evitamos prejuízo dos Maus Pagadores que o modelo pegou)
prejuizo_evitado = df_strategy.filter((col("label") == 1) & (col("decisao_final") == "NEGAR")) \
                              .selectExpr("sum(credit_amount * 0.3)").collect()[0][0]

print(f"--- RESULTADO FINANCEIRO ESTIMADO ---")
print(f"Prejuízo Potencial Total (Sem Modelo): ${total_prejuizo_sem_modelo:,.2f}")
print(f"Prejuízo Evitado pelo Modelo:          ${prejuizo_evitado:,.2f}")
print(f"Eficiência Financeira:                 {(prejuizo_evitado/total_prejuizo_sem_modelo)*100:.1f}%")

In [0]:
# Célula 4: Distribuição dos Segmentos
import matplotlib.pyplot as plt
import seaborn as sns

# Converter para Pandas para plotar
pdf_strat = df_strategy.select("segmento_risco", "credit_amount").toPandas()

plt.figure(figsize=(10, 6))
sns.countplot(x="segmento_risco", data=pdf_strat, palette="Blues_d", order=["1. Baixo Risco (VIP)", "2. Risco Médio", "3. Alto Risco"])
plt.title("Distribuição da Carteira de Clientes por Risco", fontsize=15)
plt.xlabel("Segmento", fontsize=12)
plt.ylabel("Quantidade de Clientes", fontsize=12)

plt.show()

In [0]:
# Investigando os Erros Críticos (Falsos Negativos)
# Onde o modelo disse "Aprova" (Prediction=0) mas o cliente era "Caloteiro" (Label=1)

# Puxamos os dados da tabela Gold
df_analise = spark.read.table("portfolio_credit.german_credit_gold_predictions")

# Filtramos apenas os erros graves (Prejuízo para o banco)
erros_graves = df_analise.filter((col("label") == 1) & (col("prediction") == 0))

print(f"Total de erros graves (Aprovamos quem não devia): {erros_graves.count()}")

# Vamos ver as estatísticas desses erros
erros_graves.select("age", "credit_amount", "duration", "checking_status_ix").summary().show()

# Insight Visual: Será que erramos mais nos Jovens?
pdf_erro = erros_graves.select("age").toPandas()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(pdf_erro['age'], bins=20, color='red', kde=True)
plt.title("Distribuição de Idade dos 'Falsos Bons Pagadores'")
plt.xlabel("Idade")
plt.show()